# Main Chabot File 

In [19]:
import numpy as np
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
# Add any other required imports here (e.g., tokenizer, label encoder)

# Load the model and other necessary utilities
model = load_model('chatbot_model.keras')



In [21]:
# Load the tokenizer and encoder from pickle files
import pickle
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
    
with open('encoder.pickle', 'rb') as enc:
    encoder = pickle.load(enc)
    

In [22]:
max_sequence_len = max([len(x) for x in X])

def chatbot_response(text):
    # Tokenize and pad the user input
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_len, padding='post')
    
    # Predict the intent
    prediction = model.predict(padded_sequence)
    intent_index = np.argmax(prediction, axis=1)[0]
    intent = encoder.inverse_transform([intent_index])[0]

    # Generate a response (You'll need to create a mechanism to map intents to responses)
    response = generate_response(intent)
    
    return response

def generate_response(intent):
    # Dummy response generator - replace with your own logic
    responses = {
        "greeting": "Hello! How can I help you?",
        "goodbye": "Goodbye! Have a great day!",
        # Add more intents and responses here
    }
    return responses.get(intent, "Sorry, I didn't understand that.")

# Main chat loop
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break
    print("Bot:", chatbot_response(user_input))

ValueError: in user code:

    File "D:\Users\dagbo\OneDrive\ChatbotAE2\.venv\lib\site-packages\keras\src\engine\training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "D:\Users\dagbo\OneDrive\ChatbotAE2\.venv\lib\site-packages\keras\src\engine\training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "D:\Users\dagbo\OneDrive\ChatbotAE2\.venv\lib\site-packages\keras\src\engine\training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "D:\Users\dagbo\OneDrive\ChatbotAE2\.venv\lib\site-packages\keras\src\engine\training.py", line 2381, in predict_step
        return self(x, training=False)
    File "D:\Users\dagbo\OneDrive\ChatbotAE2\.venv\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "D:\Users\dagbo\OneDrive\ChatbotAE2\.venv\lib\site-packages